In [1]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

In [2]:
# define the problem setup and environment

L = 16
K = 2
n_deltas = 5
w1 = 0.2
delta = 0.075
w2 = w1 - delta
T = 100000

w_arr = np.zeros((L, ))

for j in range(L):
    if j < K:
        w_arr[j] = w1
    else:
        w_arr[j] = w2

# print(w_mat)

In [3]:
def calc_regret(selected_items, w_arr, K):
    rew_arr = 1 - w_arr
    optimal_set = rew_arr[:K]
    selected_set = rew_arr[selected_items]
    optimal_reward = 1 - np.prod(optimal_set)
    obtained_reward = 1 - np.prod(selected_set)
    return optimal_reward - obtained_reward

In [4]:
# define the ts-cascade algorithm 
def ts_cascade(L, K, w_arr, T):
    mu_arr = np.zeros((L, ))
    v_arr = np.zeros((L, ))
    N_arr = np.zeros((L, ))
    theta_arr = np.zeros((L, ))
    sigma_arr = np.zeros((L, ))
    regret_array = np.zeros((T, ))
    curr_regret = 0
    
    for t in tqdm(range(1, T+1), desc="TS-CASCADE"):
        Z = np.random.normal(loc=0, scale=1, size=(1, ))[0]
        
        v_arr = mu_arr * (1 - mu_arr)
        term1 = np.sqrt(v_arr*np.log(t+1)/(N_arr+1))
        term2 = np.log(t+1)/(N_arr+1)
        sigma_arr = np.max(np.vstack((term1, term2)), axis=0)
        theta_arr = mu_arr + (Z*sigma_arr)
            
        selected_items = np.flip(np.argsort(theta_arr, kind='mergesort'))[:K]
        curr_regret += calc_regret(selected_items, w_arr, K)
        regret_array[t-1] = curr_regret
        
        for i in selected_items:
            W = 0
            a = np.random.uniform(low=0, high=1, size=(1, ))[0]
            if a <= w_arr[i]:
                W = 1
            
            mu_arr[i] = ((N_arr[i]*mu_arr[i]) + W)/(N_arr[i] + 1)
            N_arr[i] = N_arr[i] + 1
    
    return regret_array

In [5]:
# define cascade-ucb algorithm
def cascade_ucb(L, K, w_arr, T):
    mu_arr = np.zeros((L, ))
    N_arr = np.ones((L, ))
    UCB_arr = np.zeros((L, ))
    regret_array = np.zeros((T, ))
    curr_regret = 0
    
    for i in range(L):
        W = 0
        a = np.random.uniform(low=0, high=1, size=(1, ))[0]
        if a <= w_arr[i]:
            W = 1
        
        mu_arr[i] = W
    
    for t in tqdm(range(1, T+1), desc="CASACDE-UCB"):
        UCB_arr = mu_arr + np.sqrt(1.5*np.log(t)/N_arr)
        
        selected_items = np.flip(np.argsort(UCB_arr, kind='mergesort'))[:K]
        curr_regret += calc_regret(selected_items, w_arr, K)
        regret_array[t-1] = curr_regret
        
        for i in selected_items:
            W = 0
            a = np.random.uniform(low=0, high=1, size=(1, ))[0]
            if a <= w_arr[i]:
                W = 1
                mu_arr[i] = ((N_arr[i]*mu_arr[i]) + W)/(N_arr[i] + 1)
                N_arr[i] = N_arr[i] + 1
                break
            else:
                mu_arr[i] = ((N_arr[i]*mu_arr[i]) + W)/(N_arr[i] + 1)
                N_arr[i] = N_arr[i] + 1
    
    return regret_array

In [6]:
# define cascade-klucb algorithm
def cascade_klucb(L, K, w_arr, T):
    mu_arr = np.zeros((L, ))
    N_arr = np.ones((L, ))
    UCB_arr = np.zeros((L, ))
    regret_array = np.zeros((T, ))
    curr_regret = 0
    
    for i in range(L):
        W = 0
        a = np.random.uniform(low=0, high=1, size=(1, ))[0]
        if a <= w_arr[i]:
            W = 1
        
        mu_arr[i] = W
    
    for t in tqdm(range(1, T+1), desc="CASCADE-KLUCB"):
        UCB_arr = mu_arr + KLUCB(mu_arr, N_arr, t, L)
        
        selected_items = np.flip(np.argsort(UCB_arr, kind='mergesort'))[:K]
        curr_regret += calc_regret(selected_items, w_arr, K)
        regret_array[t-1] = curr_regret
        
        for i in selected_items:
            W = 0
            a = np.random.uniform(low=0, high=1, size=(1, ))[0]
            if a <= w_arr[i]:
                W = 1
                mu_arr[i] = ((N_arr[i]*mu_arr[i]) + W)/(N_arr[i] + 1)
                N_arr[i] = N_arr[i] + 1
                break
            else:
                mu_arr[i] = ((N_arr[i]*mu_arr[i]) + W)/(N_arr[i] + 1)
                N_arr[i] = N_arr[i] + 1
    
    return regret_array

def KLUCB(mu_arr, N_arr, t, L):
    KLUCB_arr = np.zeros((L, ))
    for i in range(L):
        p = mu_arr[i]
        N = N_arr[i]
        q_arr = np.arange(start=p, stop=1, step=1e-2)
        KLUCB_UB = np.log(t) + (3*np.log(np.log(t)))
        KLUCB_prev = 0
        KLUCB_curr = KLUCB_prev
        q_prev = p
        for q in q_arr:
            D_KL = (p*np.log(p/q)) + ((1-p)*np.log((1-p)/(1-q)))
            KLUCB_curr = N*D_KL
            if KLUCB_curr > KLUCB_UB:
                break
            KLUCB_prev = KLUCB_curr
            q_prev = q
        KLUCB_arr[i] = q_prev
    return KLUCB_arr

In [7]:
def theo_ts_cascade(L, K, T):
    T_arr = np.arange(T)+1
    term1 = 6*np.sqrt(K*L*T_arr)*np.log(T_arr) 
    term2 = 144*L*np.power(np.log(T_arr), 2.5)
    return term1 + term2

In [8]:
# n_exps = 20
# exp_regrets_1 = np.zeros((n_exps, n_deltas))
# exp_regrets_2 = np.zeros((n_exps, n_deltas))
# exp_regrets_3 = np.zeros((n_exps, T))

# for exp in range(n_exps):
#         regret_array_2 = cascade_ucb(L, K, w_arr, T)
#     regret_array_3 = ts_cascade(L, K, w_arr, T)
#         regret_array_1 = cascade_klucb(L, K, w_arr, T)
#         exp_regrets_1[exp][i] = regret_array_1[-1]
#         exp_regrets_2[exp][i] = regret_array_2[-1]
#     exp_regrets_3[exp] = regret_array_3

# avg_regret_1 = np.mean(exp_regrets_1, axis=0)
# var_regret_1 = np.mean(np.power(exp_regrets_1 - avg_regret_1, 2), axis=0)
# std_regret_1 = np.power(var_regret_1, 0.5)

# avg_regret_2 = np.mean(exp_regrets_2, axis=0)
# var_regret_2 = np.mean(np.power(exp_regrets_2 - avg_regret_2, 2), axis=0)
# std_regret_2 = np.power(var_regret_2, 0.5)

# avg_regret_3 = np.mean(exp_regrets_3, axis=0)
# var_regret_3 = np.mean(np.power(exp_regrets_3 - avg_regret_3, 2), axis=0)
# std_regret_3 = np.power(var_regret_3, 0.5)

In [9]:
# plt.errorbar(deltas, avg_regret_1, yerr=std_regret_1, c='red', fmt='none')
# plt.plot(deltas, avg_regret_1, c='red', label='CASCADE-KLUCB')
# plt.errorbar(deltas, avg_regret_2, yerr=std_regret_2, c='blue', fmt='none')
# plt.plot(deltas, avg_regret_2, c='blue', label='CASCADE-UCB')
# plt.errorbar(np.arange(T), avg_regret_3, yerr=std_regret_3, c='green', fmt='none')
# plt.plot(np.arange(T), avg_regret_3, c='green', label='EXP-REGRET')
# plt.plot(np.arange(T), theo_regret_3, c='green', label='THEO-UB', linestyle='dashed')
# plt.xlabel(r"T $\longrightarrow$")
# plt.ylabel(r"Reg(T) $\longrightarrow$")
# plt.title(r"Actual v/s Theoretical for TS-CASCADE Algo.")
# plt.legend()
# plt.show()